In [1]:
df = spark.sql("SELECT * FROM tfdeproject_fabric_lakehouse.tbl_latest_news")
display(df)

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0adc91d1-bfab-4b1f-ab3d-3da6afeddd99)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 4, Finished, Available, Finished)

In [3]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 5, Finished, Available, Finished)

In [4]:
#Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 6, Finished, Available, Finished)

In [5]:
display(result)

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f0a83395-1e17-443c-8a4c-236dea37a384)

In [6]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 8, Finished, Available, Finished)

In [7]:
display(sentiment_df)

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 53cef7e7-b177-43e1-a486-08e9ed78514b)

In [8]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 10, Finished, Available, Finished)

In [9]:
display(sentiment_df_final)

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 617f6f76-8051-43d3-bb0a-b84358f81a16)

In [10]:
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 12, Finished, Available, Finished)

In [11]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'tfdeproject_fabric_lakehouse.tbl_sentiment_analysis'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view

                    ON source_view.url = target_table.url
                    
                    WHEN MATCHED AND 
                    source_view.title <> target_table.title OR
                    source_view.link <> target_table.link OR
                    source_view.source <> target_table.source OR
                    source_view.datePublished <> target_table.datePublished OR
                    source_view.description <> target_table.description OR
                    source_view.thumbnail <> target_table.thumbnail   

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 13, Finished, Available, Finished)

In [12]:
display(sentiment_df_final)

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 312e08cf-3169-41a0-8cb5-5c2584dd6507)

In [13]:
%%sql

SELECT count(*) from tfdeproject_fabric_lakehouse.tbl_sentiment_analysis

StatementMeta(, bb6a287c-28a4-4ad6-a9db-9397f6a187ce, 15, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>